# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに参加するためのエージェントを作成することを目的としています。特に、「20の質問」ゲームにおける質問者（推測者）と回答者エージェントを実装しています。

### 問題の内容
ノートブックは、「20問ゲーム」における推測者と回答者のエージェントを作成し、ゲーム環境を設定、管理、そしてエピソードの進行を追跡します。このゲームでは、質問者が「はい」または「いいえ」で答える質問を投げかけ、できるだけ早くターゲットワードを推測することを目指します。

### 使用している手法とライブラリ
- **ライブラリ**:
  - `pygame`: ゲーム開発やマルチメディアアプリケーションのためのライブラリ。
  - `kaggle_environments`: Kaggleのコンペ環境を構築するためのライブラリ。
  
- **手法**:
  1. **キーワードの処理**: 提供されたファイルからキーワードを読み込み、JSON形式で解析し、カテゴリごとに整理します。
  2. **エージェントのロジック**:
     - **推測者エージェント**: 質問を構築し、過去の質問や確認したカテゴリに基づいて新たな質問を行い、回答の履歴を管理します。
     - **回答者エージェント**: 質問に対して「はい」または「いいえ」で回答し、確定したカテゴリを記録します。
  3. **エピソードの実行**: 環境を作成し、指定回数エピソードを実行して、全体のメトリクス（合計報酬、成功率、平均報酬など）を収集します。
  4. **データの視覚化**: 最後のエピソードを描画して、ゲームの進行状況を可視化します。

このノートブックは、ゲームアルゴリズムの実装とエージェントの行動の最適化を通じて、ゲームプレイの効果を測定し、改善するための基盤を提供します。また、エピソードの結果を基にエージェントの性能を評価し、次回の改善に生かすことを可能にします。

---


# 用語概説 
以下に、初心者がつまずきそうな専門用語を簡単に解説します。

### 専門用語解説

1. **pygameライブラリ**:
   - ゲームやマルチメディアアプリケーションを作成するためのPythonライブラリです。グラフィックス、音声、イベント処理などの機能があり、ゲーム制作を簡単に行えるようにしています。

2. **kaggle_environmentsライブラリ**:
   - Kaggleコンペティションのシミュレーション環境を構築するためのライブラリです。複数のエージェント（AI）を作成し、対戦させたり、環境を管理したりするのに役立ちます。

3. **JSON (JavaScript Object Notation)**:
   - データを構造化して保存するための軽量なフォーマットで、人間にも読みやすく、機械も簡単に解析できます。データ交換によく使われます。

4. **メモリ構造**:
   - エージェントが過去の質問、回答、カテゴリなどを記録し、後の判断に活用するためのデータ構造です。これにより、エージェントはゲームの進行に応じた情報を蓄積できます。

5. **インタラクション**:
   - AIエージェント同士が行うやり取りのことです。このコンテキストでは、質問をしたり答えたりするプロセスを指します。

6. **エピソード**:
   - 環境での一回の試行のことを指します。コンペティションのゲームが1回終了するまでの過程を指しており、エピソードごとにエージェントのパフォーマンスを評価します。

7. **メトリクス**:
   - モデルやエージェントの性能を評価するための指標です。ここでは、合計報酬、エピソード数、成功数、平均報酬、成功率が含まれます。

8. **推測者エージェント**:
   - 「20の質問」ゲームにおける質問を行い、回答を元に進行する役割を持つエージェントです。

9. **答え手エージェント**:
   - 質問者の問いに対して「はい」または「いいえ」で答える役割を持つエージェントです。

10. **キーワード**:
    - ゲーム内で推測するための重要な単語やフレーズ。システムがこれらのキーワードを使って質問を生成したり、推測を行ったりします。

これらの用語は、初心者にとってはあまり馴染みのないかもしれないものであり、特にKaggleのような競技的な環境での使用が多いものです。

---


In [ ]:
# pygameライブラリをインストールします。
# pygameはゲーム制作やマルチメディアアプリケーションの開発に便利なライブラリです。
!pip install pygame

In [ ]:
# kaggle_environmentsライブラリをインストールします。
# このライブラリはKaggleのコンペティション環境を構築するために使用されます。
# -qオプションは出力を抑制し、-Uオプションは既存のライブラリをアップグレードします。
!pip install -q -U kaggle_environments

In [ ]:
import json
import random
from kaggle_environments import make

# 提供されたファイルからキーワードを読み込みます
with open('/kaggle/input/llm-20-questions/llm_20_questions/keywords.py') as file:
    file_content = file.read()

# JSON部分を抽出します
start_index = file_content.find('[')
end_index = file_content.rfind(']') + 1
keywords_json_str = file_content[start_index:end_index]

# JSONを解析します
keywords = json.loads(keywords_json_str)

# カテゴリごとにキーワードを整理します
keyword_dict = {}
for category_data in keywords:
    category = category_data["category"]
    words = category_data["words"]
    # キーワードとその代替語を辞書として保存
    keyword_dict[category] = {word["keyword"]: word["alts"] for word in words}

# 答え手が考えている例のキーワードを選びます
answerer_category = random.choice(list(keyword_dict.keys()))
answerer_word = random.choice(list(keyword_dict[answerer_category].keys()))

# インタラクションを追跡するためのメモリ構造
guesser_memory = {
    "questions": [],  # 質問の履歴
    "answers": [],    # 回答の履歴
    "categories": []  # 確認したカテゴリの履歴
}

# メトリクスの追跡
metrics = {
    "total_reward": 0,  # 合計報酬
    "episodes": 0,      # エピソードの数
    "successes": 0      # 成功の数
}

# 推測者エージェントのロジック
def guesser_agent(observation, configuration):
    if observation["turnType"] == "ask":
        # 確認されていないカテゴリを選びます
        possible_categories = list(keyword_dict.keys())
        if guesser_memory["categories"]:
            possible_categories = [cat for cat in possible_categories if cat not in guesser_memory["categories"]]

        if not possible_categories:
            possible_categories = list(keyword_dict.keys())

        # 質問を構築します
        category = random.choice(possible_categories)
        question = f"それは{category}に関連していますか？"
        guesser_memory["questions"].append(question)  # 質問をメモリに追加
        return question

    elif observation["turnType"] == "guess":
        # 確認されたカテゴリに基づいて推測します
        confirmed_category = None
        if "yes" in guesser_memory["answers"]:
            confirmed_category = guesser_memory["categories"][guesser_memory["answers"].index("yes")]
        if not confirmed_category:
            confirmed_category = random.choice(list(keyword_dict.keys()))

        guess = random.choice(list(keyword_dict[confirmed_category].keys()))
        guesser_memory["questions"].append(guess)  # 推測をメモリに追加
        return guess

# 答え手エージェントのロジック
def answerer_agent(observation, configuration):
    if observation["turnType"] == "answer":
        question = observation["questions"][-1]  # 最後の質問を取得
        category = question.split()[-1][:-1]  # 質問からカテゴリを抽出
        guesser_memory["categories"].append(category)  # カテゴリをメモリに追加
        if category == answerer_category:
            answer = "yes"  # 正しい場合は「はい」
        else:
            answer = "no"   # 違う場合は「いいえ」
        guesser_memory["answers"].append(answer)  # 回答をメモリに追加
        return answer

# 環境で必要なエージェント関数を定義
def agent(observation, configuration):
    if observation["role"] == "guesser":
        return guesser_agent(observation, configuration)  # 推測者エージェントを呼び出す
    else:
        return answerer_agent(observation, configuration)  # 答え手エージェントを呼び出す

# 環境を作成します
env = make("llm_20_questions", debug=True)

# メトリクスを収集するために複数のエピソードを実行します
num_episodes = 100
for episode in range(num_episodes):
    env.reset()  # 環境をリセット
    steps = env.run([agent, agent, agent, agent])  # エージェントを実行
    
    # 報酬と成功を計算します
    reward = sum(step[0]['reward'] for step in steps)
    metrics["total_reward"] += reward  # 合計報酬を更新
    metrics["episodes"] += 1  # エピソード数を更新
    if steps[-1][0]['status'] == 'SUCCESS':
        metrics["successes"] += 1  # 成功数を更新

# エピソードごとの平均報酬を計算します
metrics["average_reward"] = metrics["total_reward"] / metrics["episodes"]
metrics["success_rate"] = metrics["successes"] / metrics["episodes"]

print("Metrics:", metrics)  # メトリクスを表示

# ゲームを視覚化するために最後のエピソードを描画します
env.render(mode="ipython", width=800, height=700)

# コンペティションにファイルを提出する

In [ ]:
# 提供されたコードを使用してsubmission.pyというPythonスクリプトを作成します
script_content = """
import json
import random
from kaggle_environments import make

# 提供されたファイルからキーワードを読み込みます
with open('/kaggle/input/llm-20-questions/llm_20_questions/keywords.py') as file:
    file_content = file.read()

# JSON部分を抽出します
start_index = file_content.find('[')
end_index = file_content.rfind(']') + 1
keywords_json_str = file_content[start_index:end_index]

# JSONを解析します
keywords = json.loads(keywords_json_str)

# カテゴリごとにキーワードを整理します
keyword_dict = {}
for category_data in keywords:
    category = category_data["category"]
    words = category_data["words"]
    keyword_dict[category] = {word["keyword"]: word["alts"] for word in words}

# 答え手が考えている例のキーワードを選びます
answerer_category = random.choice(list(keyword_dict.keys()))
answerer_word = random.choice(list(keyword_dict[answerer_category].keys()))

# インタラクションを追跡するためのメモリ構造
guesser_memory = {
    "questions": [],
    "answers": [],
    "categories": []
}

# メトリクスの追跡
metrics = {
    "total_reward": 0,
    "episodes": 0,
    "successes": 0
}

# 推測者エージェントのロジック
def guesser_agent(observation, configuration):
    if observation["turnType"] == "ask":
        # 確認されていないカテゴリを選びます
        possible_categories = list(keyword_dict.keys())
        if guesser_memory["categories"]:
            possible_categories = [cat for cat in possible_categories if cat not in guesser_memory["categories"]]

        if not possible_categories:
            possible_categories = list(keyword_dict.keys())

        # 質問を構築します
        category = random.choice(possible_categories)
        question = f"それは{category}に関連していますか？"
        guesser_memory["questions"].append(question)  # 質問をメモリに追加
        return question

    elif observation["turnType"] == "guess":
        # 確認されたカテゴリに基づいて推測します
        confirmed_category = None
        if "yes" in guesser_memory["answers"]:
            confirmed_category = guesser_memory["categories"][guesser_memory["answers"].index("yes")]
        if not confirmed_category:
            confirmed_category = random.choice(list(keyword_dict.keys()))

        guess = random.choice(list(keyword_dict[confirmed_category].keys()))
        guesser_memory["questions"].append(guess)  # 推測をメモリに追加
        return guess

# 答え手エージェントのロジック
def answerer_agent(observation, configuration):
    if observation["turnType"] == "answer":
        question = observation["questions"][-1]  # 最後の質問を取得
        category = question.split()[-1][:-1]  # 質問からカテゴリを抽出
        guesser_memory["categories"].append(category)  # カテゴリをメモリに追加
        if category == answerer_category:
            answer = "yes"  # 正しい場合は「はい」
        else:
            answer = "no"   # 違う場合は「いいえ」
        guesser_memory["answers"].append(answer)  # 回答をメモリに追加
        return answer

# 環境で必要なエージェント関数を定義
def agent(observation, configuration):
    if observation["role"] == "guesser":
        return guesser_agent(observation, configuration)  # 推測者エージェントを呼び出す
    else:
        return answerer_agent(observation, configuration)  # 答え手エージェントを呼び出す

# 環境を作成します
env = make("llm_20_questions", debug=True)

# メトリクスを収集するために複数のエピソードを実行します
num_episodes = 100
for episode in range(num_episodes):
    env.reset()  # 環境をリセット
    steps = env.run([agent, agent, agent, agent])  # エージェントを実行
    
    # 報酬と成功を計算します
    reward = sum(step[0]['reward'] for step in steps)
    metrics["total_reward"] += reward  # 合計報酬を更新
    metrics["episodes"] += 1  # エピソード数を更新
    if steps[-1][0]['status'] == 'SUCCESS':
        metrics["successes"] += 1  # 成功数を更新

# エピソードごとの平均報酬を計算します
metrics["average_reward"] = metrics["total_reward"] / metrics["episodes"]
metrics["success_rate"] = metrics["successes"] / metrics["episodes"]

print("Metrics:", metrics)  # メトリクスを表示

# ゲームを視覚化するために最後のエピソードを描画します
env.render(mode="ipython", width=800, height=700)
"""

# スクリプトの内容をsubmission.pyに書き込みます
submission_file_path = '/kaggle/working/submission.py'
with open(submission_file_path, 'w') as file:
    file.write(script_content)

submission_file_path  # 作成したファイルのパスを返します

---

# コメント

> ## d1v1s10n_by_zer0
> 
> 素晴らしい仕事です！
> 
> 
> 


---